### Load Envs 

In [3]:
import os
from reasoning_engines.langchain_llm_wrappers import QuickAzureChatOpenAI, QuickAzureOpenAIClient
from tools.get_tools import DefaultSchemasTools
from langchain import hub
from utils.load_envs import Environ
from chains.qa_evaluators import CustomQAEvaluator


### ReAct

In [ ]:
import os
from reasoning_engines.langchain_llm_wrappers import QuickAzureChatOpenAI, QuickAzureOpenAIClient
from tools.get_tools import DefaultSchemasTools
from langchain import hub
from utils.load_envs import Environ
from chains.qa_evaluators import CustomQAEvaluator


#### ReAct Agent
from agents.react_agents import ReActAgent

GPT_VER = '0613'
react_agent = ReActAgent(reasoninig_engine=QuickAzureChatOpenAI(version=GPT_VER), 
                        base_prompt="hwchase17/react-multi-input-json", 
                        schemas_and_tools=DefaultSchemasTools(azure_gpt_version=GPT_VER), 
                        evaluator=CustomQAEvaluator(llm=QuickAzureChatOpenAI(version='1106')), 
                        stop_words=["Observation"], 
                        thought_word= 'Thought', 
                        action_word='Action',
                        file_logging=True,
                        horizon=6)


react_agent.clear_logs()
react_agent.run_agent_trials(num_trials=2, 
query="Who is older? The former president of Korea vs. the MLB American League MVP of the last season. What is the result if you sum up the ages of the two?",
reference="The former former president of Korea is older. The sum of their age is 99.")



### Reflexion ReAct

In [ ]:
import os
from reasoning_engines.langchain_llm_wrappers import QuickAzureChatOpenAI, QuickAzureOpenAIClient
from tools.get_tools import DefaultSchemasTools
from langchain import hub
from utils.load_envs import Environ
from chains.qa_evaluators import CustomQAEvaluator


#### Reflexion Agent
from agents.reflexion_react_agents import ReflexionReActAgent
from chains.reflexion_chains import BaseReflexionChain
from fewshot_examples.reflections import REACT_REFLECTION_HEADER, REACT_REFLECTIONS

GPT_VER = '0613'   
reflexion_chains = BaseReflexionChain(
                    reasoninig_engine=QuickAzureChatOpenAI(version=GPT_VER),
                    prompt="jet-taekyo-lee/experimental-reflextion", 
                    schemas_and_tools=DefaultSchemasTools(azure_gpt_version=GPT_VER),
                    reflexion_examples=REACT_REFLECTIONS
                    )

reflexion_prompt = "jet-taekyo-lee/reflexion-react"
reflexion_agent = ReflexionReActAgent(
                        reflexion_chain = reflexion_chains,
                        reflexion_header = REACT_REFLECTION_HEADER,
                        reasoninig_engine=QuickAzureChatOpenAI(version=GPT_VER), 
                        base_prompt="hwchase17/react-multi-input-json", 
                        schemas_and_tools=DefaultSchemasTools(azure_gpt_version=GPT_VER), 
                        evaluator=CustomQAEvaluator(llm=QuickAzureChatOpenAI(version='1106')), 
                        stop_words=["Observation"], 
                        thought_word= 'Thought', 
                        action_word='Action',
                        file_logging=True,
                        horizon=6)



reflexion_agent.clear_logs()
reflexion_agent.run_agent_trials(num_trials=2, 
query="Who is older? The former president of Korea vs. the MLB American League MVP of the last season. What is the result if you sum up the ages of the two?",
reference="The former former president of Korea is older. The sum of their age is 99.")



[INFO] Query: Who is older? The former president of Korea vs. the MLB American League MVP of the last season. What is the result if you sum up the ages of the two?
[INFO] Trial 1
[INFO] Thought 1: I need to find out who the former president of Korea is and who the MLB American League MVP of the last season is. I'll start with the former president of Korea.
[INFO] Action 1: YDCSearch(query='former president of Korea')
[INFO] Observation 1: The most recent former president of South Korea is Moon Jae-In, who served from 2017 to 2022.
[INFO] Thought 2: Now I need to find out Moon Jae-In's age.
[INFO] Action 2: YDCSearch(query='Moon Jae-In age')
[INFO] Observation 2: Moon Jae-in was born on January 24, 1953.
[INFO] Thought 3: I need to calculate Moon Jae-In's age based on his birth year. I'll use the current year to do this.
[INFO] Action 3: GetDatetime(IANA_timezone='Asia/Seoul')
[INFO] Observation 3: timezone: Asia/Seoul, current time(year-month-day-hour-minute): 2023-11-30-14-14
[INFO] T

### OpenAI FunctionCalling agent

In [ ]:
import os
from reasoning_engines.langchain_llm_wrappers import QuickAzureChatOpenAI, QuickAzureOpenAIClient
from tools.get_tools import DefaultSchemasTools
from langchain import hub
from utils.load_envs import Environ
from chains.qa_evaluators import CustomQAEvaluator


# #### OepnAI Agent
GPT_VER = '0613' 
from agents.openai_function_calling_agent import OpenAIFuntionCallingAgent
openai_agent = OpenAIFuntionCallingAgent(
    reasoninig_engine = QuickAzureChatOpenAI(version=GPT_VER),
    base_prompt = "jet-taekyo-lee/openai-function-calling-agent",
    schemas_and_tools=DefaultSchemasTools(azure_gpt_version=GPT_VER),
    evaluator = CustomQAEvaluator(llm=QuickAzureChatOpenAI(version='1106')),
    action_word = 'Invoking',
    horizon = 3)

openai_agent.clear_logs()
openai_agent.run_agent_trials(num_trials=2, 
query="Who is older? The former president of Korea vs. the MLB American League MVP of the last season. What is the result if you sum up the ages of the two?",
reference="The former former president of Korea is older. The sum of their age is 99.")

[INFO] Query: Who is older? The former president of Korea vs. the MLB American League MVP of the last season. What is the result if you sum up the ages of the two?
[INFO] Trial 1
[INFO] Invoking 1: YDCSearch(query='former president of Korea age')
[INFO] Observation 1: The former president of South Korea, Moon Jae-in, was born on January 24, 1953. As of 2022, he is 69 years old. The current president, Yoon Suk Yeol, was born on December 18, 1960, making him 62 years old as of 2022.
[INFO] Invoking 2: YDCSearch(query='MLB American League MVP last season age')
[INFO] Observation 2: The American League MVP for the 2023 season was Shohei Ohtani from the Los Angeles Angels. He was born on July 5, 1994, which would have made him 29 years old during the 2023 season.
[INFO] Invoking 3: PythonREPL(command='print(69 + 29)')
Python REPL can execute arbitrary code. Use with caution.
[INFO] Observation 3: 98
[INFO] Jugdement: You failed to provide an answer because you exceeded the permitted number 

### Reflexion OpenAI FunctionCalling agent

In [ ]:
import os
from reasoning_engines.langchain_llm_wrappers import QuickAzureChatOpenAI, QuickAzureOpenAIClient
from tools.get_tools import DefaultSchemasTools
from langchain import hub
from utils.load_envs import Environ
from chains.qa_evaluators import CustomQAEvaluator


# # #### Reflexion OepnAI Agent
from agents.reflexion_openai_function_calling_agent import ReflexionOpenAIFuntionCallingAgent
from chains.reflexion_chains import BaseReflexionChain
from fewshot_examples.reflections import OPENAI_REFLECTIONS, REACT_REFLECTION_HEADER

GPT_VER = '0613'    
openai_reflexion_chains = BaseReflexionChain(
                    reasoninig_engine=QuickAzureChatOpenAI(version=GPT_VER),
                    prompt="jet-taekyo-lee/openai-reflexion-chain-prompt", 
                    schemas_and_tools=DefaultSchemasTools(azure_gpt_version=GPT_VER),
                    reflexion_examples=OPENAI_REFLECTIONS
                    )


openai_reflexion_agent = ReflexionOpenAIFuntionCallingAgent(
                        reflexion_chain = openai_reflexion_chains,
                        reflexion_header = REACT_REFLECTION_HEADER,
                        reasoninig_engine=QuickAzureChatOpenAI(version=GPT_VER), 
                        base_prompt="jet-taekyo-lee/rexlexion-openai-function-calling-agent", 
                        schemas_and_tools=DefaultSchemasTools(azure_gpt_version=GPT_VER),
                        evaluator=CustomQAEvaluator(llm=QuickAzureChatOpenAI(version='1106')),
                        action_word='Invoking',
                        file_logging=True,
                        horizon = 2)

openai_reflexion_agent.clear_logs()
openai_reflexion_agent.run_agent_trials(num_trials=2, 
query="Who is older? The former president of Korea vs. the MLB American League MVP of the last season. What is the result if you sum up the ages of the two?",
reference="The former former president of Korea is older. The sum of their age is 99.")


[INFO] Query: Who is older? The former president of Korea vs. the MLB American League MVP of the last season. What is the result if you sum up the ages of the two?
[INFO] Trial 1
[INFO] Invoking 1: YDCSearch(query='former president of Korea age')
[INFO] Observation 1: The former president of South Korea, Moon Jae-in, was born on January 24, 1953. As of 2022, he is 69 years old. The current president, Yoon Suk Yeol, was born on December 18, 1960, making him 62 years old as of 2022.
[INFO] Invoking 2: YDCSearch(query='MLB American League MVP last season age')
[INFO] Observation 2: The American League MVP for the 2023 season was Shohei Ohtani. He was born on July 5, 1994, which would have made him 29 years old during the 2023 season.
[INFO] Jugdement: You failed to provide an answer because you exceeded the permitted number of reasoning steps. You must give an answer within 2 steps.
[INFO] Trial 2
[INFO] Reflexion......
[INFO] You have attempted to answer following question before and fai

### Parallel Function Calling agent

In [ ]:
import os
from reasoning_engines.langchain_llm_wrappers import QuickAzureChatOpenAI, QuickAzureOpenAIClient
from tools.get_tools import DefaultSchemasTools
from langchain import hub
from utils.load_envs import Environ
from chains.qa_evaluators import CustomQAEvaluator


#### Parallel Function Calling Agent
GPT_VER = '1106' 
from agents.parallel_func_calling_agent import OpenAIParallelFuntionCallingAgent
parallel_calling_agent = OpenAIParallelFuntionCallingAgent(
    reasoninig_engine = QuickAzureChatOpenAI(version=GPT_VER),
    base_prompt = 'jet-taekyo-lee/parallel-function-calling-agent',
    schemas_and_tools = DefaultSchemasTools(azure_gpt_version=GPT_VER),
    evaluator = CustomQAEvaluator(llm=QuickAzureChatOpenAI('0613')),
    action_word = 'Tool invocations in parallel',
    observation_word = 'Tool-invocations results in parallel',
    use_chat_completion_api = True,
    azure_apenai_client = QuickAzureOpenAIClient(GPT_VER),
    horizon = 5
)





parallel_calling_agent.clear_logs()
parallel_calling_agent.run_agent_trials(num_trials=2, 
query="Who is older? The former president of Korea vs. the MLB American League MVP of the last season. What is the result if you sum up the ages of the two?",
reference="The former former president of Korea is older. The sum of their age is 99.")



### Reflexion + Parallel Function Calling agent

In [1]:
import os
from reasoning_engines.langchain_llm_wrappers import QuickAzureChatOpenAI, QuickAzureOpenAIClient
from tools.get_tools import DefaultSchemasTools
from langchain import hub
from utils.load_envs import Environ
from chains.qa_evaluators import CustomQAEvaluator


#### Reflexion + Parallel Function Calling Agent
from chains.reflexion_chains import BaseReflexionChain
from agents.reflexion_parallel_calling_agent import ReflexionOpenAIParallelFuntionCallingAgent
from fewshot_examples.reflections import OPENAI_PARALLEL_CALLING_REFLECTIONS, REACT_REFLECTION_HEADER

GPT_VER = '1106'    
openai_parallel_calling_reflexion_chains = BaseReflexionChain(
                    reasoninig_engine=QuickAzureChatOpenAI(version=GPT_VER),
                    prompt="jet-taekyo-lee/openai-parallel-calling-reflexion-chain-prompt", 
                    schemas_and_tools=DefaultSchemasTools(azure_gpt_version=GPT_VER),
                    reflexion_examples=OPENAI_PARALLEL_CALLING_REFLECTIONS
                    )


openai_parallel_calling_reflexion_agent = ReflexionOpenAIParallelFuntionCallingAgent(
                        reflexion_chain = openai_parallel_calling_reflexion_chains,
                        reflexion_header = REACT_REFLECTION_HEADER,
                        reasoninig_engine=QuickAzureChatOpenAI(version=GPT_VER), 
                        base_prompt="jet-taekyo-lee/reflexion-openai-parallel-calling-agent", 
                        schemas_and_tools=DefaultSchemasTools(azure_gpt_version=GPT_VER),
                        evaluator=CustomQAEvaluator(llm=QuickAzureChatOpenAI(version='1106')),
                        action_word = 'Tool invocations in parallel',
                        observation_word = 'Tool-invocations results in parallel',
                        use_chat_completion_api = True,
                        azure_apenai_client = QuickAzureOpenAIClient(GPT_VER),
                        file_logging=True,
                        horizon = 3)


openai_parallel_calling_reflexion_agent.clear_logs()
# parallel_calling_agent.run_agent_trials(num_trials=3, 
# query="Which is bigger? The summation of the current Korean president's age and Joe Biden'age vs. the summation of current temperatures of Seoul, Tokyo, and New York?",
# reference="The summation of the current Korean president's age and Joe Biden'age is bigger.")
openai_parallel_calling_reflexion_agent.run_agent_trials(num_trials=2, 
query="Who is older? The former president of Korea vs. the MLB American League MVP of the last season. What is the result if you sum up the ages of the two?",
reference="The former former president of Korea is older. The sum of their age is 99.")



[INFO] Query: Who is older? The former president of Korea vs. the MLB American League MVP of the last season. What is the result if you sum up the ages of the two?
[INFO] Trial 1
[INFO] (step 1-action) Tool invocations in parallel: [Tool 1-> YDCSearch(query=former president of South Korea), Tool 2-> YDCSearch(query=MLB American League MVP last season)]
[INFO] (step 1-observation) :: [Tool 1-> Park Geun-hye is a former president of South Korea who served as the 11th president from 2013 until her impeachment in 2017. She was the first woman to be elected as president of South Korea and the first female president popularly elected as head of state in East Asia. Her presidency ended when she was impeached and subsequently convicted on charges related to corruption. After spending nearly five years in prison, Park Geun-hye was released in December 2021 following a presidential pardon., Tool 2-> The American League MVP for the 2023 season was Shohei Ohtani of the LA Angels.]
[INFO] (step 2-a

RateLimitError for _get_function_observation. -----> Will automatically retry 20 seconds later.
20 19 18 17 16 15 14 13 12 11 10 9 8 7 6 5 4 3 2 1 RateLimitError for _get_function_observation. -----> Will automatically retry 20 seconds later.
20 19 18 17 16 15 14 13 12 11 10 9 8 7 6 5 4 3 2 1 

[INFO] (step 2-observation) :: [Tool 1-> Park Geun-hye was born on February 2, 1952. As of my knowledge cutoff in 2023, she would be 71 years old., Tool 2-> Shohei Ohtani is 29 years old.]
[INFO] (step 3-action) Tool invocations in parallel: [Tool 1-> PythonREPL(command=print(71 + 29))]
Python REPL can execute arbitrary code. Use with caution.
[INFO] (step 3-observation) :: [Tool 1-> 100]
[INFO] Jugdement: You failed to provide an answer because you exceeded the permitted number of reasoning steps. You must give an answer within 3 steps.
[INFO] Trial 2
[INFO] Reflexion......
[INFO] You have attempted to answer following question before and failed. The following reflection(s) give a plan to avoid failing to answer the question in the same way you did previously. Use them to improve your strategy of correctly answering the given question.
- Reflection: The previous trial failed because the answer was not provided within the permitted number of reasoning steps. The process of identifying 

RateLimitError for get_chat_response_tool_use. -----> Will automatically retry 20 seconds later.
20 19 18 17 16 15 14 13 12 11 10 9 8 7 6 5 4 3 2 1 RateLimitError for get_chat_response_tool_use. -----> Will automatically retry 20 seconds later.
20 19 18 17 16 15 14 13 12 11 10 9 8 7 6 5 4 3 2 1 

[INFO] (step 2-action) Tool invocations in parallel: [Tool 1-> YDCSearch(query=Park Geun-hye age), Tool 2-> YDCSearch(query=Shohei Ohtani age)]


RateLimitError for _get_function_observation. -----> Will automatically retry 20 seconds later.
20 19 18 17 16 15 14 13 12 11 10 9 8 7 6 5 4 3 2 1 

[INFO] (step 2-observation) :: [Tool 1-> Park Geun-hye was born on February 2, 1952. As of my knowledge cutoff in 2023, she would be 71 years old., Tool 2-> Shohei Ohtani is 29 years old.]
[INFO] (step 3-action) Tool invocations in parallel: [Tool 1-> PythonREPL(command=print(71 + 29, 71 > 29))]
[INFO] (step 3-observation) :: [Tool 1-> 100 True]
[INFO] Jugdement: You failed to provide an answer because you exceeded the permitted number of reasoning steps. You must give an answer within 3 steps.
